# NetworkInteraction

In [1]:
#export

import base64
from typing import Optional

from algosdk.future.transaction import SignedTransaction
from algosdk.v2client import algod


class NetworkInteraction:
    @staticmethod
    def wait_for_confirmation(client: algod.AlgodClient, txid, log=True):
        """
        Utility function to wait until the transaction is
        confirmed before proceeding.
        """
        print('1:NetworkInteraction.wait_for_confirmation')
        last_round = client.status().get('last-round')
        txinfo = client.pending_transaction_info(txid)
        while not (txinfo.get('confirmed-round') and txinfo.get('confirmed-round') > 0):
            if log:
                print("Waiting for confirmation")
            print('1:NetworkInteraction.wait_for_confirmation', {'last_round': last_round, 'txinfo': txinfo})
            last_round += 1
            client.status_after_block(last_round)
            txinfo = client.pending_transaction_info(txid)
        if log:
            print(f"Transaction {txid} confirmed in round {txinfo.get('confirmed-round')}.")
        return txinfo

    @staticmethod
    def get_default_suggested_params(client: algod.AlgodClient):
        """
        Gets default suggested params with flat transaction fee and fee amount of 1000.
        :param client:
        :return:
        """
        print('1:NetworkInteraction.get_default_suggested_params')
        suggested_params = client.suggested_params()

        suggested_params.flat_fee = True
        suggested_params.fee = 1000

        return suggested_params

    @staticmethod
    def submit_asa_creation(client: algod.AlgodClient, transaction: SignedTransaction) -> Optional[int]:
        """
        Submits a ASA creation transaction to the network. If the transaction is successful the ASA's id is returned.
        :param client:
        :param transaction:
        :return:
        """
        print('1:NetworkInteraction.submit_asa_creation')
        txid = client.send_transaction(transaction)

        NetworkInteraction.wait_for_confirmation(client, txid)

        try:
            ptx = client.pending_transaction_info(txid)
            return ptx["asset-index"]
        except Exception as e:
            # TODO: Proper logging needed.
            print(e)
            print('Unsuccessful creation of Algorand Standard Asset.')

    @staticmethod
    def submit_transaction(client: algod.AlgodClient, transaction: SignedTransaction, log=True) -> Optional[str]:
        print('1:NetworkInteraction.submit_transaction')
        txid = client.send_transaction(transaction)

        NetworkInteraction.wait_for_confirmation(client, txid, log)

        return txid

    @staticmethod
    def compile_program(client: algod.AlgodClient, source_code):
        """
        :param client: algorand client
        :param source_code: teal source code
        :return:
            Decoded byte program
        """
        print('1:NetworkInteraction.compile_program')
        compile_response = client.compile(source_code)
        return base64.b64decode(compile_response['result'])

# ApplicationTransactionRepository

In [2]:
#export

import base64
from algosdk.v2client import algod
from algosdk.future import transaction as algo_txn
from typing import List, Any, Optional, Union
from algosdk import account as algo_acc
from algosdk.future.transaction import Transaction, SignedTransaction


def get_default_suggested_params(client: algod.AlgodClient):
    """
    Gets default suggested params with flat transaction fee and fee amount of 1000.
    :param client:
    :return:
    """
    print('2:get_default_suggested_params')
    suggested_params = client.suggested_params()

    suggested_params.flat_fee = True
    suggested_params.fee = 1000

    return suggested_params


class ApplicationTransactionRepository:
    """
    Initializes transaction related to applications.
    """

    @classmethod
    def create_application(cls,
                           client: algod.AlgodClient,
                           creator_private_key: str,
                           approval_program: bytes,
                           clear_program: bytes,
                           global_schema: algo_txn.StateSchema,
                           local_schema: algo_txn.StateSchema,
                           app_args: Optional[List[Any]],
                           sign_transaction: bool = True) -> Union[Transaction, SignedTransaction]:
        """
        Initiates a transaction that represents application creation. The transaction is optionally signed using the
        provided private key.
        :param client: algorand client.
        :param creator_private_key: private key of the creator of the application.
        :param approval_program: encoded source code in bytes for the approval program.
        :param clear_program: encoded source code in bytes for the clear program.
        :param global_schema: global schema for the application.
        :param local_schema: local schema for the application.
        :param app_args: list of arguments for the application.
        :param sign_transaction: boolean value that determines whether the created transaction should be signed or not.
        :return:
            Returns SignedTransaction or Transaction depending on the boolean property sign_transaction.
        """
        print('2:ApplicationTransactionRepository.create_application')
        creator_address = algo_acc.address_from_private_key(private_key=creator_private_key)
        suggested_params = get_default_suggested_params(client=client)

        txn = algo_txn.ApplicationCreateTxn(sender=creator_address,
                                            sp=suggested_params,
                                            on_complete=algo_txn.OnComplete.NoOpOC.real,
                                            approval_program=approval_program,
                                            clear_program=clear_program,
                                            global_schema=global_schema,
                                            local_schema=local_schema,
                                            app_args=app_args)

        if sign_transaction:
            txn = txn.sign(private_key=creator_private_key)

        return txn

    @classmethod
    def call_application(cls,
                         client: algod.AlgodClient,
                         caller_private_key: str,
                         app_id: int,
                         on_complete: algo_txn.OnComplete,
                         app_args: Optional[List[Any]] = None,
                         sign_transaction: bool = True) -> Union[Transaction, SignedTransaction]:
        """
        Creates a transaction that represents an application call.
        :param client: algorand client.
        :param caller_private_key: the private key of the caller of the application.
        :param app_id: the application id which identifies the app.
        :param on_complete: Type of the application call.
        :param app_args: Arguments of the application.
        :param sign_transaction: boolean value that determines whether the created transaction should be signed or not.
        :return:
        Returns SignedTransaction or Transaction depending on the boolean property sign_transaction.
        """
        print('2:ApplicationTransactionRepository.call_application')
        caller_address = algo_acc.address_from_private_key(private_key=caller_private_key)
        suggested_params = get_default_suggested_params(client=client)

        txn = algo_txn.ApplicationCallTxn(sender=caller_address,
                                          sp=suggested_params,
                                          index=app_id,
                                          app_args=app_args,
                                          on_complete=on_complete)

        if sign_transaction:
            txn = txn.sign(private_key=caller_private_key)

        return txn


class ASATransactionRepository:
    """
    Initializes transactions related to Algorand Standard Assets
    """

    @classmethod
    def create_asa(cls,
                   client: algod.AlgodClient,
                   creator_private_key: str,
                   unit_name: str,
                   asset_name: str,
                   total: int,
                   decimals: int,
                   note: Optional[bytes] = None,
                   manager_address: Optional[str] = None,
                   reserve_address: Optional[str] = None,
                   freeze_address: Optional[str] = None,
                   clawback_address: Optional[str] = None,
                   url: Optional[str] = None,
                   default_frozen: bool = False,
                   sign_transaction: bool = True) -> Union[Transaction, SignedTransaction]:
        """
        :param client:
        :param creator_private_key:
        :param unit_name:
        :param asset_name:
        :param total:
        :param decimals:
        :param note:
        :param manager_address:
        :param reserve_address:
        :param freeze_address:
        :param clawback_address:
        :param url:
        :param default_frozen:
        :param sign_transaction:
        :return:
        """
        print('2:ASATransactionRepository.create_application')

        suggested_params = get_default_suggested_params(client=client)

        creator_address = algo_acc.address_from_private_key(private_key=creator_private_key)

        txn = algo_txn.AssetConfigTxn(sender=creator_address,
                                      sp=suggested_params,
                                      total=total,
                                      default_frozen=default_frozen,
                                      unit_name=unit_name,
                                      asset_name=asset_name,
                                      manager=manager_address,
                                      reserve=reserve_address,
                                      freeze=freeze_address,
                                      clawback=clawback_address,
                                      url=url,
                                      decimals=decimals,
                                      note=note)

        if sign_transaction:
            txn = txn.sign(private_key=creator_private_key)

        return txn

    @classmethod
    def create_non_fungible_asa(cls,
                                client: algod.AlgodClient,
                                creator_private_key: str,
                                unit_name: str,
                                asset_name: str,
                                note: Optional[bytes] = None,
                                manager_address: Optional[str] = None,
                                reserve_address: Optional[str] = None,
                                freeze_address: Optional[str] = None,
                                clawback_address: Optional[str] = None,
                                url: Optional[str] = None,
                                default_frozen: bool = False,
                                sign_transaction: bool = True) -> Union[Transaction, SignedTransaction]:
        """
        :param client:
        :param creator_private_key:
        :param unit_name:
        :param asset_name:
        :param note:
        :param manager_address:
        :param reserve_address:
        :param freeze_address:
        :param clawback_address:
        :param url:
        :param default_frozen:
        :param sign_transaction:
        :return:
        """
        print('2:ASATransactionRepository.create_non_fungible_asa')

        return ASATransactionRepository.create_asa(client=client,
                                                   creator_private_key=creator_private_key,
                                                   unit_name=unit_name,
                                                   asset_name=asset_name,
                                                   total=1,
                                                   decimals=0,
                                                   note=note,
                                                   manager_address=manager_address,
                                                   reserve_address=reserve_address,
                                                   freeze_address=freeze_address,
                                                   clawback_address=clawback_address,
                                                   url=url,
                                                   default_frozen=default_frozen,
                                                   sign_transaction=sign_transaction)

    @classmethod
    def asa_opt_in(cls,
                   client: algod.AlgodClient,
                   sender_private_key: str,
                   asa_id: int,
                   sign_transaction: bool = True) -> Union[Transaction, SignedTransaction]:
        """
        Opts-in the sender's account to the specified asa with an id: asa_id.
        :param client:
        :param sender_private_key:
        :param asa_id:
        :param sign_transaction:
        :return:
        """
        print('2:ASATransactionRepository.asa_opt_in')

        suggested_params = get_default_suggested_params(client=client)
        sender_address = algo_acc.address_from_private_key(sender_private_key)

        txn = algo_txn.AssetTransferTxn(sender=sender_address,
                                        sp=suggested_params,
                                        receiver=sender_address,
                                        amt=0,
                                        index=asa_id)

        if sign_transaction:
            txn = txn.sign(private_key=sender_private_key)

        return txn

    @classmethod
    def asa_transfer(cls,
                     client: algod.AlgodClient,
                     sender_address: str,
                     receiver_address: str,
                     asa_id: int,
                     amount: int,
                     revocation_target: Optional[str],
                     sender_private_key: Optional[str],
                     sign_transaction: bool = True) -> Union[Transaction, SignedTransaction]:
        """
        :param client:
        :param sender_address:
        :param receiver_address:
        :param asa_id:
        :param amount:
        :param revocation_target:
        :param sender_private_key:
        :param sign_transaction:
        :return:
        """
        print('2:ASATransactionRepository.asa_transfer')
        suggested_params = get_default_suggested_params(client=client)

        txn = algo_txn.AssetTransferTxn(sender=sender_address,
                                        sp=suggested_params,
                                        receiver=receiver_address,
                                        amt=amount,
                                        index=asa_id,
                                        revocation_target=revocation_target)

        if sign_transaction:
            txn = txn.sign(private_key=sender_private_key)

        return txn

    @classmethod
    def change_asa_management(cls,
                              client: algod.AlgodClient,
                              current_manager_pk: str,
                              asa_id: int,
                              manager_address: Optional[str] = None,
                              reserve_address: Optional[str] = None,
                              freeze_address: Optional[str] = None,
                              clawback_address: Optional[str] = None,
                              strict_empty_address_check: bool = True,
                              sign_transaction: bool = True) -> Union[Transaction, SignedTransaction]:
        """
        Changes the management properties of a given ASA.
        :param client:
        :param current_manager_pk:
        :param asa_id:
        :param manager_address:
        :param reserve_address:
        :param freeze_address:
        :param clawback_address:
        :param strict_empty_address_check:
        :param sign_transaction:
        :return:
        """
        print('2:ASATransactionRepository.change_asa_management')

        params = get_default_suggested_params(client=client)

        current_manager_address = algo_acc.address_from_private_key(private_key=current_manager_pk)

        txn = algo_txn.AssetConfigTxn(
            sender=current_manager_address,
            sp=params,
            index=asa_id,
            manager=manager_address,
            reserve=reserve_address,
            freeze=freeze_address,
            clawback=clawback_address,
            strict_empty_address_check=strict_empty_address_check)

        if sign_transaction:
            txn = txn.sign(private_key=current_manager_pk)

        return txn


class PaymentTransactionRepository:

    @classmethod
    def payment(cls,
                client: algod.AlgodClient,
                sender_address: str,
                receiver_address: str,
                amount: int,
                sender_private_key: Optional[str],
                sign_transaction: bool = True) -> Union[Transaction, SignedTransaction]:
        """
        Creates a payment transaction in ALGOs.
        :param client:
        :param sender_address:
        :param receiver_address:
        :param amount:
        :param sender_private_key:
        :param sign_transaction:
        :return:
        """
        print('2:PaymentTransactionRepository.payment')
        suggested_params = get_default_suggested_params(client=client)

        txn = algo_txn.PaymentTxn(sender=sender_address,
                                  sp=suggested_params,
                                  receiver=receiver_address,
                                  amt=amount)

        if sign_transaction:
            txn = txn.sign(private_key=sender_private_key)

        return txn

# game_funds_escrow

In [3]:
#export

from pyteal import *


def game_funds_escrow(app_id: int):
    print('3:game_funds_escrow')

    win_refund = Seq([
        Assert(Gtxn[0].application_id() == Int(app_id)),
        Assert(Gtxn[1].fee() <= Int(1000)),
        Assert(Gtxn[1].asset_close_to() == Global.zero_address()),
        Assert(Gtxn[1].rekey_to() == Global.zero_address())
    ])

    tie_refund = Seq([
        Assert(Gtxn[0].application_id() == Int(app_id)),
        Assert(Gtxn[1].fee() <= Int(1000)),
        Assert(Gtxn[1].asset_close_to() == Global.zero_address()),
        Assert(Gtxn[1].rekey_to() == Global.zero_address()),
        Assert(Gtxn[2].fee() <= Int(1000)),
        Assert(Gtxn[2].asset_close_to() == Global.zero_address()),
        Assert(Gtxn[2].rekey_to() == Global.zero_address())
    ])

    return Seq([
        Cond(
            [Global.group_size() == Int(2), win_refund],
            [Global.group_size() == Int(3), tie_refund],
        ),
        Return(Int(1))
    ])

In [4]:
import main

timeout_seconds = 90
approval_program = lambda: main.arrow_1(main.US0(1, timeout_seconds))

def clear_program():
    print('5:clear_program')
    return Return(Int(1))

def number_of_int():
    print('5:AppVariables.number_of_int')
    return 5

def number_of_str():
    print('5:AppVariables.number_of_str')
    return 4

main.spi Loaded


# GameEngineService

In [5]:
#export

from algosdk import logic as algo_logic
from algosdk.future import transaction as algo_txn
from pyteal import compileTeal, Mode


class GameEngineService:
    """
    Engine that defines the interaction and initialization of the Tic-Tac-Toe DApp.
    """

    def __init__(self,
                 app_creator_pk,
                 app_creator_address,
                 player_x_pk,
                 player_x_address,
                 player_o_pk,
                 player_o_address):
        print('4:GameEngineService.__init__')

        self.app_creator_pk = app_creator_pk
        self.app_creator_address = app_creator_address
        self.player_x_pk = player_x_pk
        self.player_x_address = player_x_address
        self.player_o_pk = player_o_pk
        self.player_o_address = player_o_address
        self.teal_version = 4

        self.approval_program_code = approval_program()
        self.clear_program_code = clear_program()

        self.app_id = None
        self.escrow_fund_address = None
        self.escrow_fund_program_bytes = None

    def deploy_application(self, client):
        """
        Creates and sends the transaction to the network that does the initialization of the Tic-Tac-Toe game.
        :param client:
        :return:
        """
        print('4:GameEngineService.deploy_application')
        approval_program_compiled = compileTeal(approval_program(),
                                                mode=Mode.Application,
                                                version=self.teal_version)

        clear_program_compiled = compileTeal(clear_program(),
                                             mode=Mode.Application,
                                             version=self.teal_version)

        approval_program_bytes = NetworkInteraction.compile_program(client=client,
                                                                    source_code=approval_program_compiled)

        clear_program_bytes = NetworkInteraction.compile_program(client=client,
                                                                 source_code=clear_program_compiled)

        global_schema = algo_txn.StateSchema(num_uints=number_of_int(),
                                             num_byte_slices=number_of_str())

        local_schema = algo_txn.StateSchema(num_uints=0,
                                            num_byte_slices=0)

        app_transaction = ApplicationTransactionRepository.create_application(client=client,
                                                                              creator_private_key=self.app_creator_pk,
                                                                              approval_program=approval_program_bytes,
                                                                              clear_program=clear_program_bytes,
                                                                              global_schema=global_schema,
                                                                              local_schema=local_schema,
                                                                              app_args=None)

        tx_id = NetworkInteraction.submit_transaction(client,
                                                      transaction=app_transaction,
                                                      log=False)

        transaction_response = client.pending_transaction_info(tx_id)

        self.app_id = transaction_response['application-index']
        print(f"Tic-Tac-Toe application deployed with the application_id: {self.app_id}")

        return f"Tic-Tac-Toe application deployed with the application_id: {self.app_id}"

    def start_game(self, client):
        """
        Atomic transfer of 3 transactions:
        - 1. Application call
        - 2. Payment from the Player X address to the Escrow fund address
        - 3. Payment from the Player O address to the Escrow fund address
        :param client:
        :return:
        """
        print('4:GameEngineService.start_game')
        if self.app_id is None:
            raise ValueError('The application has not been deployed')

        if self.escrow_fund_address is not None or self.escrow_fund_program_bytes is not None:
            raise ValueError('The game has already started!')

        escrow_fund_program_compiled = compileTeal(game_funds_escrow(app_id=self.app_id),
                                                   mode=Mode.Signature,
                                                   version=self.teal_version)

        self.escrow_fund_program_bytes = NetworkInteraction.compile_program(client=client,
                                                                            source_code=escrow_fund_program_compiled)

        self.escrow_fund_address = algo_logic.address(self.escrow_fund_program_bytes)

        player_x_funding_txn = PaymentTransactionRepository.payment(client=client,
                                                                    sender_address=self.player_x_address,
                                                                    receiver_address=self.escrow_fund_address,
                                                                    amount=1000000,
                                                                    sender_private_key=None,
                                                                    sign_transaction=False)

        player_o_funding_txn = PaymentTransactionRepository.payment(client=client,
                                                                    sender_address=self.player_o_address,
                                                                    receiver_address=self.escrow_fund_address,
                                                                    amount=1000000,
                                                                    sender_private_key=None,
                                                                    sign_transaction=False)

        app_args = [
            "SetupPlayers"
        ]

        app_initialization_txn = \
            ApplicationTransactionRepository.call_application(client=client,
                                                              caller_private_key=self.app_creator_pk,
                                                              app_id=self.app_id,
                                                              on_complete=algo_txn.OnComplete.NoOpOC,
                                                              app_args=app_args,
                                                              sign_transaction=False)

        gid = algo_txn.calculate_group_id([app_initialization_txn,
                                           player_x_funding_txn,
                                           player_o_funding_txn])

        app_initialization_txn.group = gid
        player_x_funding_txn.group = gid
        player_o_funding_txn.group = gid

        app_initialization_txn_signed = app_initialization_txn.sign(self.app_creator_pk)
        player_x_funding_txn_signed = player_x_funding_txn.sign(self.player_x_pk)
        player_o_funding_txn_signed = player_o_funding_txn.sign(self.player_o_pk)

        signed_group = [app_initialization_txn_signed,
                        player_x_funding_txn_signed,
                        player_o_funding_txn_signed]

        txid = client.send_transactions(signed_group)

        print(f"Game started with the transaction_id: {txid}")

        return f"Game started with the transaction_id: {txid}"

    def play_action(self, client, player_id: str, action_position: int):
        """
        Application call transaction that performs an action for the specified player at the specified action position.
        :param client:
        :param player_id: "X" or "O"
        :param action_position: action position in the range of [0, 8]
        :return:
        """
        print('4:GameEngineService.play_action')
        if player_id != "X" and player_id != "O":
            raise ValueError('Invalid player id! The player_id should be X or O.')

        if self.app_id is None:
            raise ValueError('The application has not been deployed')

        app_args = [
            "ActionMove",
            action_position]

        player_pk = self.player_x_pk if player_id == "X" else self.player_o_pk

        app_initialization_txn = \
            ApplicationTransactionRepository.call_application(client=client,
                                                              caller_private_key=player_pk,
                                                              app_id=self.app_id,
                                                              on_complete=algo_txn.OnComplete.NoOpOC,
                                                              app_args=app_args)

        tx_id = NetworkInteraction.submit_transaction(client,
                                                      transaction=app_initialization_txn,
                                                      log=False)

        print(f"{player_id} has been put at position {action_position} in transaction with id: {tx_id}")

        return f"{player_id} has been put at position {action_position} in transaction with id: {tx_id}"

    def fund_escrow(self, client):
        """
        Funding the escrow address in order to handle the transactions fees for refunding.
        :param client:
        :return:
        """
        print('4:GameEngineService.fund_escrow')
        fund_escrow_txn = PaymentTransactionRepository.payment(client=client,
                                                               sender_address=self.app_creator_address,
                                                               receiver_address=self.escrow_fund_address,
                                                               amount=150000,
                                                               sender_private_key=self.app_creator_pk,
                                                               sign_transaction=True)

        tx_id = NetworkInteraction.submit_transaction(client,
                                                      transaction=fund_escrow_txn,
                                                      log=False)

        print(f'Escrow address has been funded in transaction with id: {tx_id}')
        return f'Escrow address has been funded in transaction with id: {tx_id}'

    def win_money_refund(self, client, player_id: str):
        """
        Atomic transfer of 2 transactions:
        1. Application call
        2. Payment from the Escrow account to winner address either PlayerX or PlayerO.
        :param client:
        :param player_id: "X" or "O".
        :return:
        """
        print('4:GameEngineService.win_money_refund')
        if player_id != "X" and player_id != "O":
            raise ValueError('Invalid player id! The player_id should be X or O.')

        if self.app_id is None:
            raise ValueError('The application has not been deployed')

        player_pk = self.player_x_pk if player_id == "X" else self.player_o_pk
        player_address = self.player_x_address if player_id == "X" else self.player_o_address

        app_args = [
            "MoneyRefund"
        ]

        app_withdraw_call_txn = \
            ApplicationTransactionRepository.call_application(client=client,
                                                              caller_private_key=player_pk,
                                                              app_id=self.app_id,
                                                              on_complete=algo_txn.OnComplete.NoOpOC,
                                                              app_args=app_args,
                                                              sign_transaction=False)

        refund_txn = PaymentTransactionRepository.payment(client=client,
                                                          sender_address=self.escrow_fund_address,
                                                          receiver_address=player_address,
                                                          amount=2000000,
                                                          sender_private_key=None,
                                                          sign_transaction=False)

        gid = algo_txn.calculate_group_id([app_withdraw_call_txn,
                                           refund_txn])

        app_withdraw_call_txn.group = gid
        refund_txn.group = gid

        app_withdraw_call_txn_signed = app_withdraw_call_txn.sign(player_pk)

        refund_txn_logic_signature = algo_txn.LogicSig(self.escrow_fund_program_bytes)
        refund_txn_signed = algo_txn.LogicSigTransaction(refund_txn, refund_txn_logic_signature)

        signed_group = [app_withdraw_call_txn_signed,
                        refund_txn_signed]

        txid = client.send_transactions(signed_group)

        print(f"The winning money have been refunded to the player {player_id} in the transaction with id: {txid}")
        return f"The winning money have been refunded to the player {player_id} in the transaction with id: {txid}"

    def tie_money_refund(self, client):
        """
        Atomic transfer of 3 transactions:
        1. Application call
        2. Payment from the escrow address to the PlayerX address.
        3. Payment from the escrow address to the PlayerO address.
        :param client:
        :return:
        """
        print('4:GameEngineService.tie_money_refund')
        if self.app_id is None:
            raise ValueError('The application has not been deployed')

        app_args = [
            "MoneyRefund"
        ]

        app_withdraw_call_txn = \
            ApplicationTransactionRepository.call_application(client=client,
                                                              caller_private_key=self.app_creator_pk,
                                                              app_id=self.app_id,
                                                              on_complete=algo_txn.OnComplete.NoOpOC,
                                                              app_args=app_args,
                                                              sign_transaction=False)

        refund_player_x_txn = PaymentTransactionRepository.payment(client=client,
                                                                   sender_address=self.escrow_fund_address,
                                                                   receiver_address=self.player_x_address,
                                                                   amount=1000000,
                                                                   sender_private_key=None,
                                                                   sign_transaction=False)

        refund_player_o_txn = PaymentTransactionRepository.payment(client=client,
                                                                   sender_address=self.escrow_fund_address,
                                                                   receiver_address=self.player_o_address,
                                                                   amount=1000000,
                                                                   sender_private_key=None,
                                                                   sign_transaction=False)

        gid = algo_txn.calculate_group_id([app_withdraw_call_txn,
                                           refund_player_x_txn,
                                           refund_player_o_txn])

        app_withdraw_call_txn.group = gid
        refund_player_x_txn.group = gid
        refund_player_o_txn.group = gid

        app_withdraw_call_txn_signed = app_withdraw_call_txn.sign(self.app_creator_pk)

        refund_player_x_txn_logic_signature = algo_txn.LogicSig(self.escrow_fund_program_bytes)
        refund_player_x_txn_signed = \
            algo_txn.LogicSigTransaction(refund_player_x_txn, refund_player_x_txn_logic_signature)

        refund_player_o_txn_logic_signature = algo_txn.LogicSig(self.escrow_fund_program_bytes)
        refund_player_o_txn_signed = \
            algo_txn.LogicSigTransaction(refund_player_o_txn, refund_player_o_txn_logic_signature)

        signed_group = [app_withdraw_call_txn_signed,
                        refund_player_x_txn_signed,
                        refund_player_o_txn_signed]

        txid = client.send_transactions(signed_group)

        print(f"The initial bet money have been refunded to the players in the transaction with id: {txid}")

        return f"The initial bet money have been refunded to the players in the transaction with id: {txid}"

# testnet

In [6]:
# https://bank.testnet.algorand.network

token = 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'
address = "http://localhost:4001"

def get_client():
    return algod.AlgodClient(token, address, headers={'X-Api-key': token})


In [7]:
accounts = [
 {'private_key': 'OBVzhzxAK4ZPdaF1QbkXsd8hgRUPJeCfYFjFghgMGnLmcpaAD0mdQJ3Rwq/RwklKGQWAbaIP9zb5BUmcZM4DOg==',
  'address': '4ZZJNAAPJGOUBHORYKX5DQSJJIMQLADNUIH7ONXZAVEZYZGOAM5CM7HV2M',
  'mnemonic': 'poem slush dry adult final thunder price pause concert topic mesh wild dry gate judge celery winter list promote clump country loan move about faith'},
 {'private_key': 'lHFQVWKpvRYU00yAQqHOHJegn5nvP0owYkroBFKr94qGmASQPz4KrmlSbpmNeevb+v+eqDR1owwGEmFARGRDOQ==',
  'address': 'Q2MAJEB7HYFK42KSN2MY26PL3P5P7HVIGR22GDAGCJQUARDEIM4WLHSIFQ',
  'mnemonic': 'crater dove click normal kitten aisle obvious error exotic lunar guard impact donkey gun lamp divide announce match cement excess piano turn fish above make'},
 {'private_key': '0DNKDAK5QfWuMCTMNJ+Re5mSlc7rbI0q1UOUYDmvPGpVTJc2PAknfOwS1rk+MAkE5ynA1XbqLeAGlYGA1H9T4w==',
  'address': 'KVGJONR4BETXZ3AS224T4MAJATTSTQGVO3VC3YAGSWAYBVD7KPR23RKNBI',
  'mnemonic': 'key churn alert mother lock run lyrics category office lamp silver nut cement pistol fury hollow faculty feed capable announce index cloth faint about bid'}
]

accounts

[{'private_key': 'OBVzhzxAK4ZPdaF1QbkXsd8hgRUPJeCfYFjFghgMGnLmcpaAD0mdQJ3Rwq/RwklKGQWAbaIP9zb5BUmcZM4DOg==',
  'address': '4ZZJNAAPJGOUBHORYKX5DQSJJIMQLADNUIH7ONXZAVEZYZGOAM5CM7HV2M',
  'mnemonic': 'poem slush dry adult final thunder price pause concert topic mesh wild dry gate judge celery winter list promote clump country loan move about faith'},
 {'private_key': 'lHFQVWKpvRYU00yAQqHOHJegn5nvP0owYkroBFKr94qGmASQPz4KrmlSbpmNeevb+v+eqDR1owwGEmFARGRDOQ==',
  'address': 'Q2MAJEB7HYFK42KSN2MY26PL3P5P7HVIGR22GDAGCJQUARDEIM4WLHSIFQ',
  'mnemonic': 'crater dove click normal kitten aisle obvious error exotic lunar guard impact donkey gun lamp divide announce match cement excess piano turn fish above make'},
 {'private_key': '0DNKDAK5QfWuMCTMNJ+Re5mSlc7rbI0q1UOUYDmvPGpVTJc2PAknfOwS1rk+MAkE5ynA1XbqLeAGlYGA1H9T4w==',
  'address': 'KVGJONR4BETXZ3AS224T4MAJATTSTQGVO3VC3YAGSWAYBVD7KPR23RKNBI',
  'mnemonic': 'key churn alert mother lock run lyrics category office lamp silver nut cement pistol fury h

In [8]:
import algosdk

# # indexer = get_indexer()

acc_pk, acc_address = algosdk.account.generate_account()
player_x_pk, player_x_address = algosdk.account.generate_account()
player_o_pk, player_o_address = algosdk.account.generate_account()

accounts2 = [
    {'private_key': acc_pk, 'address': acc_address, 'mnemonic': algosdk.mnemonic.from_private_key(acc_pk)},
    {'private_key': player_x_pk, 'address': player_x_address, 'mnemonic': algosdk.mnemonic.from_private_key(player_x_pk)},
    {'private_key': player_o_pk, 'address': player_o_address, 'mnemonic': algosdk.mnemonic.from_private_key(player_o_pk)}
]

In [9]:
game_actions = {
    'O': [
        ("X", 0),
        ("O", 2),
        ("X", 4),
        ("O", 8),
        ("X", 6),
        ("O", 5)
    ],
    'X': [
        ("X", 0),
        ("O", 1),
        ("X", 2),
        ("O", 5),
        ("X", 4),
        ("O", 8),
        ("X", 6)
    ],
    'tie': [
        ("X", 0),
        ("O", 1),
        ("X", 2),
        ("O", 4),
        ("X", 3),
        ("O", 5),
        ("X", 7),
        ("O", 6),
        ("X", 8)
    ],
    'timeout': [
        ("X", 0),
        ("O", 2)
    ],
    'error_position_repeat': [
        ("X", 0),
        ("O", 0)
    ],
    'error_position_invalid': [
        ("X", 0),
        ("O", 9)
    ],
    'error_player_invalid': [
        ("X", 0),
        ("X", 1)
    ]
}

In [10]:
import time

def run(moves, should_revert_refund=False, reraise=False, early_stop=False, funds=1):
    start = time.time()

    client = get_client()

    game_engine_fixed = \
        GameEngineService(
            app_creator_pk=accounts[0]['private_key'],
            app_creator_address=accounts[0]['address'],
            player_x_pk=accounts[1]['private_key'],
            player_x_address=accounts[1]['address'],
            player_o_pk=accounts[2]['private_key'],
            player_o_address=accounts[2]['address']
        )

    log = {}
    log['app_deployment_txn_log'] = game_engine_fixed.deploy_application(client)
    log['start_game_txn_log'] = game_engine_fixed.start_game(client)
    error = False
    
    for player_id, action_position in game_actions[moves]:
        log_id = f'play_action_txn_log_{player_id}_{action_position}'
        
        try:
            if not error:
                log[log_id] = game_engine_fixed.play_action(
                    client=client,
                    player_id=player_id,
                    action_position=action_position
                )
        except Exception as e:
            log[f'{log_id}_error'] = e
            error = True
            if reraise:
                print(f"Total (s): {time.time() - start}")
                raise e
        
        if early_stop:
            break

    if not error:
        if moves == 'timeout':
            time.sleep(timeout_seconds + 1)

        for i in range(funds):
            log[f'fund_escrow_txn_log_{i}'] = game_engine_fixed.fund_escrow(client=client)
        
        def revert_player(player):
            if not should_revert_refund:
                return player
            else:
                return 'O' if player == 'X' else 'X'

        if moves == 'tie':
            log['tie_money_refund_txn_log'] = game_engine_fixed.tie_money_refund(client=client)
        else:
            try:
                if moves == 'timeout':
                    log['win_money_refund_txn_log'] = game_engine_fixed.win_money_refund(
                        client=client, 
                        player_id=revert_player(game_actions['timeout'][-1][0])
                    )
                else:
                    log['win_money_refund_txn_log'] = game_engine_fixed.win_money_refund(
                        client=client, 
                        player_id=revert_player(moves)
                    )
            except Exception as e:
                log['win_money_refund_txn_log_error'] = e
                if reraise:
                    print(f"Total (s): {time.time() - start}")
                    raise e

    print(f"Total (s): {time.time() - start}")

    return log

def expect_run(log, expected):
    try:
        assert list(log.keys()) == expected
    except Exception as e:
        print('log:', log)
        print('expected:', expected)
        raise e
    return log


In [11]:
expect_run(
    run('O'), 
    ['app_deployment_txn_log', 
     'start_game_txn_log', 
     'play_action_txn_log_X_0', 
     'play_action_txn_log_O_2', 
     'play_action_txn_log_X_4', 
     'play_action_txn_log_O_8', 
     'play_action_txn_log_X_6', 
     'play_action_txn_log_O_5', 
     'fund_escrow_txn_log_0', 
     'win_money_refund_txn_log']
)

4:GameEngineService.__init__
application_start
initialize_players_logic
play_action_logic
has_player_won
has_player_won
is_tie
money_refund_logic
app_initialization_logic
5:clear_program
4:GameEngineService.deploy_application
application_start
initialize_players_logic
play_action_logic
has_player_won
has_player_won
is_tie
money_refund_logic
app_initialization_logic
5:clear_program
1:NetworkInteraction.compile_program
1:NetworkInteraction.compile_program
5:AppVariables.number_of_int
5:AppVariables.number_of_str
2:ApplicationTransactionRepository.create_application
2:get_default_suggested_params
1:NetworkInteraction.submit_transaction
1:NetworkInteraction.wait_for_confirmation
1:NetworkInteraction.wait_for_confirmation {'last_round': 21862198, 'txinfo': {'pool-error': '', 'txn': {'sig': 'jZbjC3N+qUhlNcZAMLcvdZxoFhNfdO/wPLU7g+t85w9xD2hksx+K+gLHCTnHdVKZXbYYQ0oFv8t9EBc0x74CAg==', 'txn': {'apap': 'BCAMAQACwAM4B6QCkgFJkQJUAyYJDFBsYXllclhTdGF0ZQxQbGF5ZXJPU3RhdGUJR2FtZVN0YXRlDlBsYXllck9BZGRyZXN

{'app_deployment_txn_log': 'Tic-Tac-Toe application deployed with the application_id: 92487510',
 'start_game_txn_log': 'Game started with the transaction_id: J5BNRBISZY6IJRTYJD6YWSXF46J5E7RUC2Q6D2XHTTDEVBDLZR5Q',
 'play_action_txn_log_X_0': 'X has been put at position 0 in transaction with id: 5Z36LVXTIK5RCWURAIKZAL7BXVB55C2S7KMXMPOC2HYGUOHLPZ2Q',
 'play_action_txn_log_O_2': 'O has been put at position 2 in transaction with id: 6OC24SD6QFFVG2DXCAFLLFOACZIWPSNKLNYP37XDT57GEONJX6MQ',
 'play_action_txn_log_X_4': 'X has been put at position 4 in transaction with id: P7CRZBOCOPY53OVWXO4DFWEQ6JMPFON2NUQVQ7KIQNIE4VO4OXQQ',
 'play_action_txn_log_O_8': 'O has been put at position 8 in transaction with id: OSYQWFPGA4HWJ47C5TNQXEOWIPKMPNLETGXGULML6ZCK2EILA65Q',
 'play_action_txn_log_X_6': 'X has been put at position 6 in transaction with id: PEF2UTX6BGDHWQPC4FSNYWBKTVVR2HOXPXQ6KFNQUL7ZXPDKTMYA',
 'play_action_txn_log_O_5': 'O has been put at position 5 in transaction with id: MRTCSR3HN6RQ2WFWTF4

In [12]:
expect_run(
    run('X'), 
    ['app_deployment_txn_log', 
     'start_game_txn_log', 
     'play_action_txn_log_X_0', 
     'play_action_txn_log_O_1', 
     'play_action_txn_log_X_2', 
     'play_action_txn_log_O_5', 
     'play_action_txn_log_X_4', 
     'play_action_txn_log_O_8', 
     'play_action_txn_log_X_6', 
     'fund_escrow_txn_log_0', 
     'win_money_refund_txn_log']
)


4:GameEngineService.__init__
application_start
initialize_players_logic
play_action_logic
has_player_won
has_player_won
is_tie
money_refund_logic
app_initialization_logic
5:clear_program
4:GameEngineService.deploy_application
application_start
initialize_players_logic
play_action_logic
has_player_won
has_player_won
is_tie
money_refund_logic
app_initialization_logic
5:clear_program
1:NetworkInteraction.compile_program
1:NetworkInteraction.compile_program
5:AppVariables.number_of_int
5:AppVariables.number_of_str
2:ApplicationTransactionRepository.create_application
2:get_default_suggested_params
1:NetworkInteraction.submit_transaction
1:NetworkInteraction.wait_for_confirmation
1:NetworkInteraction.wait_for_confirmation {'last_round': 21862214, 'txinfo': {'pool-error': '', 'txn': {'sig': 'XhFetNjDa7goTna1wnQa3dvJ+/dExJBvDVxUzJCGrmdiYXnrMuP4HsR0u2kSlYJXef6jSOwAT0Jx55dBAixHBw==', 'txn': {'apap': 'BCAMAQACwAM4B6QCkgFJkQJUAyYJDFBsYXllclhTdGF0ZQxQbGF5ZXJPU3RhdGUJR2FtZVN0YXRlDlBsYXllck9BZGRyZXN

{'app_deployment_txn_log': 'Tic-Tac-Toe application deployed with the application_id: 92487622',
 'start_game_txn_log': 'Game started with the transaction_id: VQ4X7LEQUMW2DUO65POOS42MATSFWS26HC6MMIXBJTG5MIXMFHBQ',
 'play_action_txn_log_X_0': 'X has been put at position 0 in transaction with id: KW3PKIQLIL7POKBON6NMUJOGGF63G4LAM5ABMOIZWITH6UEPCYIA',
 'play_action_txn_log_O_1': 'O has been put at position 1 in transaction with id: G7HJUPIW7F4WRJYWFWVRGRSCSKQUUQ6DCBRJDZYKZSAXHDL5RBAQ',
 'play_action_txn_log_X_2': 'X has been put at position 2 in transaction with id: FOCQ6SADLTZ3AZSBM5J6COKM7AVT5PMP7A4BLIPHJETGMSXYSIGQ',
 'play_action_txn_log_O_5': 'O has been put at position 5 in transaction with id: KZAH37OE34GRELCPMJRCRXO5PVOF3B44NQYKBH7LUJHOFPMHFGEA',
 'play_action_txn_log_X_4': 'X has been put at position 4 in transaction with id: TPPUVQA6FLRKQG6D3VB2N33LITOMD6LMBGQT2LQV63SYSHJSNEIQ',
 'play_action_txn_log_O_8': 'O has been put at position 8 in transaction with id: CIPS6QFGEEFC6I3XQ4D

In [13]:
expect_run(
    run('tie'), 
    ['app_deployment_txn_log', 
     'start_game_txn_log', 
     'play_action_txn_log_X_0', 
     'play_action_txn_log_O_1', 
     'play_action_txn_log_X_2', 
     'play_action_txn_log_O_4', 
     'play_action_txn_log_X_3', 
     'play_action_txn_log_O_5', 
     'play_action_txn_log_X_7', 
     'play_action_txn_log_O_6', 
     'play_action_txn_log_X_8', 
     'fund_escrow_txn_log_0', 
     'tie_money_refund_txn_log']
)


4:GameEngineService.__init__
application_start
initialize_players_logic
play_action_logic
has_player_won
has_player_won
is_tie
money_refund_logic
app_initialization_logic
5:clear_program
4:GameEngineService.deploy_application
application_start
initialize_players_logic
play_action_logic
has_player_won
has_player_won
is_tie
money_refund_logic
app_initialization_logic
5:clear_program
1:NetworkInteraction.compile_program
1:NetworkInteraction.compile_program
5:AppVariables.number_of_int
5:AppVariables.number_of_str
2:ApplicationTransactionRepository.create_application
2:get_default_suggested_params
1:NetworkInteraction.submit_transaction
1:NetworkInteraction.wait_for_confirmation
1:NetworkInteraction.wait_for_confirmation {'last_round': 21862232, 'txinfo': {'pool-error': '', 'txn': {'sig': 'MTpsyaw+xA10ru6OdBymDomNgVyQCB28QbsYpqcBuer3KblhZN2E6fj8XOC5nzMkS8iGK2/DE31gp7MnzcsbBA==', 'txn': {'apap': 'BCAMAQACwAM4B6QCkgFJkQJUAyYJDFBsYXllclhTdGF0ZQxQbGF5ZXJPU3RhdGUJR2FtZVN0YXRlDlBsYXllck9BZGRyZXN

{'app_deployment_txn_log': 'Tic-Tac-Toe application deployed with the application_id: 92487739',
 'start_game_txn_log': 'Game started with the transaction_id: U77MPIEUMP6GKS6RHIIO3LSBENEOLTUKD67BIBKOFTVZDYK6OUUA',
 'play_action_txn_log_X_0': 'X has been put at position 0 in transaction with id: ZTUK7TULL2UYSN7RUKBMIA6IKDKR55FX26IDPF72Z3BJVW2D5DQQ',
 'play_action_txn_log_O_1': 'O has been put at position 1 in transaction with id: ESCD4XJYNOIYZKDMAZBLDZBFYSYAYPDGF6O4GEYYFOJXJ35DRJ5A',
 'play_action_txn_log_X_2': 'X has been put at position 2 in transaction with id: UG77TULNKBTK7T74S7MHKAOZJQMF7HYN5TI3YLAGJWRX6DE7CKQA',
 'play_action_txn_log_O_4': 'O has been put at position 4 in transaction with id: GAPZAAERCGXIGNYE46B4NXJWO476RPFERLXWRWW7Y2YFO5LZ3HUQ',
 'play_action_txn_log_X_3': 'X has been put at position 3 in transaction with id: JYAVUSQTGEUUUW6ZCEUAUUEBN5PSHAC5J2Q3T6UMX5OA7MWEGKLA',
 'play_action_txn_log_O_5': 'O has been put at position 5 in transaction with id: HPQDVUFYOMZGTZ6AIFQ

In [14]:
expect_run(
    run('timeout'), 
    ['app_deployment_txn_log', 
     'start_game_txn_log', 
     'play_action_txn_log_X_0', 
     'play_action_txn_log_O_2', 
     'fund_escrow_txn_log_0', 
     'win_money_refund_txn_log']
)


4:GameEngineService.__init__
application_start
initialize_players_logic
play_action_logic
has_player_won
has_player_won
is_tie
money_refund_logic
app_initialization_logic
5:clear_program
4:GameEngineService.deploy_application
application_start
initialize_players_logic
play_action_logic
has_player_won
has_player_won
is_tie
money_refund_logic
app_initialization_logic
5:clear_program
1:NetworkInteraction.compile_program
1:NetworkInteraction.compile_program
5:AppVariables.number_of_int
5:AppVariables.number_of_str
2:ApplicationTransactionRepository.create_application
2:get_default_suggested_params
1:NetworkInteraction.submit_transaction
1:NetworkInteraction.wait_for_confirmation
1:NetworkInteraction.wait_for_confirmation {'last_round': 21862254, 'txinfo': {'pool-error': '', 'txn': {'sig': 'lvBo45FLy5esibtKFR4QwNq9S5pPAexhodEUTdmKsl0G5T3bNr9oBXtE+wIt9oXuBlmgHcDPZsLL4d5Vxuc+Dw==', 'txn': {'apap': 'BCAMAQACwAM4B6QCkgFJkQJUAyYJDFBsYXllclhTdGF0ZQxQbGF5ZXJPU3RhdGUJR2FtZVN0YXRlDlBsYXllck9BZGRyZXN

{'app_deployment_txn_log': 'Tic-Tac-Toe application deployed with the application_id: 92487892',
 'start_game_txn_log': 'Game started with the transaction_id: NC4E66OH2P5DX6BB2ZGTIIXG676LX7UO45PORJQSBLCK24STTXSQ',
 'play_action_txn_log_X_0': 'X has been put at position 0 in transaction with id: QJNQIJXTQYPS2OGNR5GP3FMBDUMYB4D7HKQS2D4HSRP3RERI6TJA',
 'play_action_txn_log_O_2': 'O has been put at position 2 in transaction with id: QZ2T4VPKGVA2YH4SKTDXZOFU6XH5WT366IZHLWN4HS4VTUKHCKIQ',
 'fund_escrow_txn_log_0': 'Escrow address has been funded in transaction with id: A37BBL5OS35DULKYD7KZBSMUJ575K3NWLN65LXSA4U32Y4WK3BJQ',
 'win_money_refund_txn_log': 'The winning money have been refunded to the player O in the transaction with id: KMEAOGK6DHXIZDBF4V25OI2W5R6XLOCMUHAKVHDDU7NXG7NDK5GQ'}

In [15]:
expect_run(
    run('X', funds=3), 
    ['app_deployment_txn_log',
     'start_game_txn_log',
     'play_action_txn_log_X_0', 
     'play_action_txn_log_O_1', 
     'play_action_txn_log_X_2', 
     'play_action_txn_log_O_5', 
     'play_action_txn_log_X_4', 
     'play_action_txn_log_O_8', 
     'play_action_txn_log_X_6', 
     'fund_escrow_txn_log_0',
     'fund_escrow_txn_log_1',
     'fund_escrow_txn_log_2',
     'win_money_refund_txn_log']
)

4:GameEngineService.__init__
application_start
initialize_players_logic
play_action_logic
has_player_won
has_player_won
is_tie
money_refund_logic
app_initialization_logic
5:clear_program
4:GameEngineService.deploy_application
application_start
initialize_players_logic
play_action_logic
has_player_won
has_player_won
is_tie
money_refund_logic
app_initialization_logic
5:clear_program
1:NetworkInteraction.compile_program
1:NetworkInteraction.compile_program
5:AppVariables.number_of_int
5:AppVariables.number_of_str
2:ApplicationTransactionRepository.create_application
2:get_default_suggested_params
1:NetworkInteraction.submit_transaction
1:NetworkInteraction.wait_for_confirmation
1:NetworkInteraction.wait_for_confirmation {'last_round': 21862283, 'txinfo': {'pool-error': '', 'txn': {'sig': 'dsFxWecn3wstqyucORUj4namtEUwJKrsLhJPftYnBYufTvzoJAYABL0BMb2cCyoNBxL4tcHcSv6ETzi5z2UWBA==', 'txn': {'apap': 'BCAMAQACwAM4B6QCkgFJkQJUAyYJDFBsYXllclhTdGF0ZQxQbGF5ZXJPU3RhdGUJR2FtZVN0YXRlDlBsYXllck9BZGRyZXN

{'app_deployment_txn_log': 'Tic-Tac-Toe application deployed with the application_id: 92488084',
 'start_game_txn_log': 'Game started with the transaction_id: QL5XKVJMITG3MFMJ5QWQUMGFZVZGXNLMMPOI4B7NFONFZLANU2SA',
 'play_action_txn_log_X_0': 'X has been put at position 0 in transaction with id: KWBIHMFWYRVLNADL7RH34GWQVTWCZTOKSLWKSF5XHBQHOMN4AU5Q',
 'play_action_txn_log_O_1': 'O has been put at position 1 in transaction with id: CFSEKXCEULRBHTC6WQLBS6IBHGWICDAM2SCPRIIWBIVIDXYZYRQQ',
 'play_action_txn_log_X_2': 'X has been put at position 2 in transaction with id: RGR2VTPLTLTIOHS3RBX5334DRZ6SPHGE6JJH6GCIBUOWMHBVJAIQ',
 'play_action_txn_log_O_5': 'O has been put at position 5 in transaction with id: RS5RRPA4AQFPRHVQG6DQN2UWJCSUTVAAHPFUX5S35F4AVNLXSMIA',
 'play_action_txn_log_X_4': 'X has been put at position 4 in transaction with id: 3OGJZXBSUHRIANGR6RJPD7HZYOAMP7M4GUGW5RZGUTQP2UEJUT4Q',
 'play_action_txn_log_O_8': 'O has been put at position 8 in transaction with id: JEDZC6RC4BEFR2VTHGG

In [16]:
expect_run(
    run('O', funds=0), 
    ['app_deployment_txn_log', 
     'start_game_txn_log', 
     'play_action_txn_log_X_0', 
     'play_action_txn_log_O_2', 
     'play_action_txn_log_X_4', 
     'play_action_txn_log_O_8', 
     'play_action_txn_log_X_6', 
     'play_action_txn_log_O_5', 
     'win_money_refund_txn_log_error']
)


4:GameEngineService.__init__
application_start
initialize_players_logic
play_action_logic
has_player_won
has_player_won
is_tie
money_refund_logic
app_initialization_logic
5:clear_program
4:GameEngineService.deploy_application
application_start
initialize_players_logic
play_action_logic
has_player_won
has_player_won
is_tie
money_refund_logic
app_initialization_logic
5:clear_program
1:NetworkInteraction.compile_program
1:NetworkInteraction.compile_program
5:AppVariables.number_of_int
5:AppVariables.number_of_str
2:ApplicationTransactionRepository.create_application
2:get_default_suggested_params
1:NetworkInteraction.submit_transaction
1:NetworkInteraction.wait_for_confirmation
1:NetworkInteraction.wait_for_confirmation {'last_round': 21862305, 'txinfo': {'pool-error': '', 'txn': {'sig': 'O431TzG6WWX3vKPBpByCNmwB7MNOHfa6MdsBOCEZDbpa1+Y1xZvu8cHUVfZI7hyQ8iMgRn2CHYLlCMlmjk5pCw==', 'txn': {'apap': 'BCAMAQACwAM4B6QCkgFJkQJUAyYJDFBsYXllclhTdGF0ZQxQbGF5ZXJPU3RhdGUJR2FtZVN0YXRlDlBsYXllck9BZGRyZXN

{'app_deployment_txn_log': 'Tic-Tac-Toe application deployed with the application_id: 92488219',
 'start_game_txn_log': 'Game started with the transaction_id: IVVA3ZE2AZ4ZXVE5RO7Y7VV2O55LV625DTQRHTKMLKH4HYELFSYQ',
 'play_action_txn_log_X_0': 'X has been put at position 0 in transaction with id: 73TE4JDJYUIU336IEESB32BBF2XE7JAGFP4CKLI3SUOZXXJ34UMQ',
 'play_action_txn_log_O_2': 'O has been put at position 2 in transaction with id: 73EB5PZKLGVMRCQTN7HD6ZTVGHHIW3X476ONDFRROR7CMODXW7CQ',
 'play_action_txn_log_X_4': 'X has been put at position 4 in transaction with id: QV5GHBNOFLHQD6EPUXHXGRYEX7SBOJMP45TF3IT6DZPW2WKMGGAQ',
 'play_action_txn_log_O_8': 'O has been put at position 8 in transaction with id: UY5ZBYUBGVEGVGK7WGKVPELPRHIEHZRVBHKKF2XXKHN35P4NDQ5A',
 'play_action_txn_log_X_6': 'X has been put at position 6 in transaction with id: U6OCTDNXJPE4BQJ6W7DAFWDLCE4737BV6B7BTPX6HLXCQ7FX2CDQ',
 'play_action_txn_log_O_5': 'O has been put at position 5 in transaction with id: 6JZYSXNQDXK5YV3LWR7

In [17]:
expect_run(
    run('O', early_stop=True), 
    ['app_deployment_txn_log',
     'start_game_txn_log',
     'play_action_txn_log_X_0',
     'fund_escrow_txn_log_0',
     'win_money_refund_txn_log_error']
)

4:GameEngineService.__init__
application_start
initialize_players_logic
play_action_logic
has_player_won
has_player_won
is_tie
money_refund_logic
app_initialization_logic
5:clear_program
4:GameEngineService.deploy_application
application_start
initialize_players_logic
play_action_logic
has_player_won
has_player_won
is_tie
money_refund_logic
app_initialization_logic
5:clear_program
1:NetworkInteraction.compile_program
1:NetworkInteraction.compile_program
5:AppVariables.number_of_int
5:AppVariables.number_of_str
2:ApplicationTransactionRepository.create_application
2:get_default_suggested_params
1:NetworkInteraction.submit_transaction
1:NetworkInteraction.wait_for_confirmation
1:NetworkInteraction.wait_for_confirmation {'last_round': 21862319, 'txinfo': {'pool-error': '', 'txn': {'sig': 'gfjQnR/xWEzQdH6k2KkR8DFwmJEvq3B634XRMORPhNlz+iMem7mtRFeYeTzo75cwig0yHp9/5WgxfFM7iazpCA==', 'txn': {'apap': 'BCAMAQACwAM4B6QCkgFJkQJUAyYJDFBsYXllclhTdGF0ZQxQbGF5ZXJPU3RhdGUJR2FtZVN0YXRlDlBsYXllck9BZGRyZXN

{'app_deployment_txn_log': 'Tic-Tac-Toe application deployed with the application_id: 92488313',
 'start_game_txn_log': 'Game started with the transaction_id: BUEMO5BEZ62XUPKXKOC2KGECIDC3INDKSPE7SVU6DN6NVFEMO24Q',
 'play_action_txn_log_X_0': 'X has been put at position 0 in transaction with id: QY5UYHNWXTWQ67QHEOPMLAACSZRHENWXTRGBRM3GO6YKQWGEYDVQ',
 'fund_escrow_txn_log_0': 'Escrow address has been funded in transaction with id: ZX6SDYAPFKZPAUWIIDGZC6SBFZV6XN3CKLCPQGT7TL6HPHYVFN3Q',
 'win_money_refund_txn_log_error': algosdk.error.AlgodHTTPError('TransactionPool.Remember: transaction ZN327HIQIJVEGCXJ7U2QRQGDN74CRIU7GKUEYFGVHJFRXGGMFRCA: logic eval error: err opcode executed. Details: pc=296, opcodes===\nbnz label7\nerr\nlabel7:\n')}

In [18]:
expect_run(
    run('O', should_revert_refund=True), 
    ['app_deployment_txn_log', 
     'start_game_txn_log',
     'play_action_txn_log_X_0', 
     'play_action_txn_log_O_2', 
     'play_action_txn_log_X_4', 
     'play_action_txn_log_O_8', 
     'play_action_txn_log_X_6', 
     'play_action_txn_log_O_5', 
     'fund_escrow_txn_log_0', 
     'win_money_refund_txn_log_error']
)


4:GameEngineService.__init__
application_start
initialize_players_logic
play_action_logic
has_player_won
has_player_won
is_tie
money_refund_logic
app_initialization_logic
5:clear_program
4:GameEngineService.deploy_application
application_start
initialize_players_logic
play_action_logic
has_player_won
has_player_won
is_tie
money_refund_logic
app_initialization_logic
5:clear_program
1:NetworkInteraction.compile_program
1:NetworkInteraction.compile_program
5:AppVariables.number_of_int
5:AppVariables.number_of_str
2:ApplicationTransactionRepository.create_application
2:get_default_suggested_params
1:NetworkInteraction.submit_transaction
1:NetworkInteraction.wait_for_confirmation
1:NetworkInteraction.wait_for_confirmation {'last_round': 21862325, 'txinfo': {'pool-error': '', 'txn': {'sig': 'wU9OxBGFGwJDM8TcgTU6zr8YWYVW6JKdXPd8W2JF4Dw9nC0R1TwLqh8N7O8+3Sy4eCPwaisEI+tFEVVF7XxOCA==', 'txn': {'apap': 'BCAMAQACwAM4B6QCkgFJkQJUAyYJDFBsYXllclhTdGF0ZQxQbGF5ZXJPU3RhdGUJR2FtZVN0YXRlDlBsYXllck9BZGRyZXN

{'app_deployment_txn_log': 'Tic-Tac-Toe application deployed with the application_id: 92488357',
 'start_game_txn_log': 'Game started with the transaction_id: HNTU4KFSIQG2GELU5ZWXGFVPYGRAY2DOZZ3NDKSA2BW3TRU2BPIA',
 'play_action_txn_log_X_0': 'X has been put at position 0 in transaction with id: ZWMP6OEDKPCDZMI7YHXG7TBBQZDWQ6XRGKWEH5NXZCZVJROWOZCQ',
 'play_action_txn_log_O_2': 'O has been put at position 2 in transaction with id: IURXNVKFXI4LUDAY3YASFRP2GRO5EEVVBC5AREL2K6FEVRVBSNAQ',
 'play_action_txn_log_X_4': 'X has been put at position 4 in transaction with id: KLKYKQEO7CLLUZS37QERKNR6X4KDKXIH7P3OJZW5J6SKUYHU33LQ',
 'play_action_txn_log_O_8': 'O has been put at position 8 in transaction with id: RQXD5ORMG7MKHJNHSW3Z6ZBE6YIHDT6CIMECWV6FATGBPMM5KO6A',
 'play_action_txn_log_X_6': 'X has been put at position 6 in transaction with id: QLWQ4C3OEAGLMYAXG36UAYAZ2YQ4EW63TWYICW2Q5OET2S67TIAA',
 'play_action_txn_log_O_5': 'O has been put at position 5 in transaction with id: YPBYIM6R3H3SX2JP76R

In [19]:
expect_run(
    run('timeout', should_revert_refund=True), 
    ['app_deployment_txn_log', 
     'start_game_txn_log', 
     'play_action_txn_log_X_0', 
     'play_action_txn_log_O_2', 
     'fund_escrow_txn_log_0', 
     'win_money_refund_txn_log_error']
)


4:GameEngineService.__init__
application_start
initialize_players_logic
play_action_logic
has_player_won
has_player_won
is_tie
money_refund_logic
app_initialization_logic
5:clear_program
4:GameEngineService.deploy_application
application_start
initialize_players_logic
play_action_logic
has_player_won
has_player_won
is_tie
money_refund_logic
app_initialization_logic
5:clear_program
1:NetworkInteraction.compile_program
1:NetworkInteraction.compile_program
5:AppVariables.number_of_int
5:AppVariables.number_of_str
2:ApplicationTransactionRepository.create_application
2:get_default_suggested_params
1:NetworkInteraction.submit_transaction
1:NetworkInteraction.wait_for_confirmation
1:NetworkInteraction.wait_for_confirmation {'last_round': 21862341, 'txinfo': {'pool-error': '', 'txn': {'sig': 'ONKHhTEMypg183zVtFy9yJI5j5MS06BiC1f030Zv3KJjFBGaylpGuTGZUcvY6wveH8RzI8C+weWf3QMyjdg6DA==', 'txn': {'apap': 'BCAMAQACwAM4B6QCkgFJkQJUAyYJDFBsYXllclhTdGF0ZQxQbGF5ZXJPU3RhdGUJR2FtZVN0YXRlDlBsYXllck9BZGRyZXN

{'app_deployment_txn_log': 'Tic-Tac-Toe application deployed with the application_id: 92488441',
 'start_game_txn_log': 'Game started with the transaction_id: BEBON7NIC4YDESGKIOCI5KXGCLVXFDRDOB5VJTKHAA2WOCJDNFIQ',
 'play_action_txn_log_X_0': 'X has been put at position 0 in transaction with id: 63LCAHK2E3E7MFN4CJSCLZJX7HY5PNUS6OO6HY3VFK3QVADPBNHQ',
 'play_action_txn_log_O_2': 'O has been put at position 2 in transaction with id: PZ2FDTTN3I37FH2DPHNPSM5A3N6LZV3HPWD4YUUQNZ55IPGDHZPA',
 'fund_escrow_txn_log_0': 'Escrow address has been funded in transaction with id: TO333WXZYJVTCRYOBRUT3P7BDW6UHBPCIL4IV6NVLOTOKER2D3WA',
 'win_money_refund_txn_log_error': algosdk.error.AlgodHTTPError('TransactionPool.Remember: transaction EQMOYMCJAJGXCN3CZVV7ST6VSBTZGPSVG2PIIRM37QO5ZTUGDO6Q: logic eval error: assert failed pc=350. Details: pc=350, opcodes=app_global_get\n==\nassert\n')}

In [20]:
expect_run(
    run('error_position_repeat'), 
    ['app_deployment_txn_log', 
     'start_game_txn_log', 
     'play_action_txn_log_X_0', 
     'play_action_txn_log_O_0_error']
)


4:GameEngineService.__init__
application_start
initialize_players_logic
play_action_logic
has_player_won
has_player_won
is_tie
money_refund_logic
app_initialization_logic
5:clear_program
4:GameEngineService.deploy_application
application_start
initialize_players_logic
play_action_logic
has_player_won
has_player_won
is_tie
money_refund_logic
app_initialization_logic
5:clear_program
1:NetworkInteraction.compile_program
1:NetworkInteraction.compile_program
5:AppVariables.number_of_int
5:AppVariables.number_of_str
2:ApplicationTransactionRepository.create_application
2:get_default_suggested_params
1:NetworkInteraction.submit_transaction
1:NetworkInteraction.wait_for_confirmation
1:NetworkInteraction.wait_for_confirmation {'last_round': 21862370, 'txinfo': {'pool-error': '', 'txn': {'sig': 'Mpz6ggpXdnDlpqKJoVMVs1Kzu5LzBCiPfMO4bpWe6/rcTNPcjTBwO+D46QLIgYlxKUq8Mjh56KfX5tZ/3C7dDg==', 'txn': {'apap': 'BCAMAQACwAM4B6QCkgFJkQJUAyYJDFBsYXllclhTdGF0ZQxQbGF5ZXJPU3RhdGUJR2FtZVN0YXRlDlBsYXllck9BZGRyZXN

{'app_deployment_txn_log': 'Tic-Tac-Toe application deployed with the application_id: 92488605',
 'start_game_txn_log': 'Game started with the transaction_id: BEJ3EO7L6QAP6O6FRRYL5P3MOSKJ76C2CRGNCK2XBUB2CSQCP24Q',
 'play_action_txn_log_X_0': 'X has been put at position 0 in transaction with id: IRKIQAKOWRYFPV475VPOAKPEGK62NPJ5FSRPNRNYGRRDOAQQGD7Q',
 'play_action_txn_log_O_0_error': algosdk.error.AlgodHTTPError('TransactionPool.Remember: transaction 5YYQ7AUSHNC766YKRHVXQGGRPIOJYTCP7JWS7GEDK3Q5NEY7UOJA: logic eval error: assert failed pc=448. Details: pc=448, opcodes===\n&&\nassert\n')}

In [21]:
expect_run(
    run('error_position_invalid'), 
    ['app_deployment_txn_log', 
     'start_game_txn_log', 
     'play_action_txn_log_X_0', 
     'play_action_txn_log_O_9_error']
)


4:GameEngineService.__init__
application_start
initialize_players_logic
play_action_logic
has_player_won
has_player_won
is_tie
money_refund_logic
app_initialization_logic
5:clear_program
4:GameEngineService.deploy_application
application_start
initialize_players_logic
play_action_logic
has_player_won
has_player_won
is_tie
money_refund_logic
app_initialization_logic
5:clear_program
1:NetworkInteraction.compile_program
1:NetworkInteraction.compile_program
5:AppVariables.number_of_int
5:AppVariables.number_of_str
2:ApplicationTransactionRepository.create_application
2:get_default_suggested_params
1:NetworkInteraction.submit_transaction
1:NetworkInteraction.wait_for_confirmation
1:NetworkInteraction.wait_for_confirmation {'last_round': 21862374, 'txinfo': {'pool-error': '', 'txn': {'sig': '2gbVL2h5YXFlCeUvO99vlHCyCLYiZAWfPQ2r/Lbtyp/nl5/VMxwPFrbPcyG9bX0sDK+l+ug77dQMJrHqIGBPCw==', 'txn': {'apap': 'BCAMAQACwAM4B6QCkgFJkQJUAyYJDFBsYXllclhTdGF0ZQxQbGF5ZXJPU3RhdGUJR2FtZVN0YXRlDlBsYXllck9BZGRyZXN

{'app_deployment_txn_log': 'Tic-Tac-Toe application deployed with the application_id: 92488629',
 'start_game_txn_log': 'Game started with the transaction_id: J6P424IMY7UBZWKBZOP66I3TQ6I6RZ43Q32DS2YMTIRRIWGNIPOA',
 'play_action_txn_log_X_0': 'X has been put at position 0 in transaction with id: 3P4LKQSYHIKX5B42K7VTQUK4AX6OPXSWJ76JAJCFADMAI3UNVFSA',
 'play_action_txn_log_O_9_error': algosdk.error.AlgodHTTPError('TransactionPool.Remember: transaction 7PBJ5ARVMNU7NDOFSDIGWAMWT33QBS5SDJL34ZH2R2P54MWSPAKA: logic eval error: assert failed pc=405. Details: pc=405, opcodes=pushint 8\n<=\nassert\n')}

In [22]:
expect_run(
    run('error_player_invalid'), 
    ['app_deployment_txn_log', 
     'start_game_txn_log',
     'play_action_txn_log_X_0',
     'play_action_txn_log_X_1_error']
)


4:GameEngineService.__init__
application_start
initialize_players_logic
play_action_logic
has_player_won
has_player_won
is_tie
money_refund_logic
app_initialization_logic
5:clear_program
4:GameEngineService.deploy_application
application_start
initialize_players_logic
play_action_logic
has_player_won
has_player_won
is_tie
money_refund_logic
app_initialization_logic
5:clear_program
1:NetworkInteraction.compile_program
1:NetworkInteraction.compile_program
5:AppVariables.number_of_int
5:AppVariables.number_of_str
2:ApplicationTransactionRepository.create_application
2:get_default_suggested_params
1:NetworkInteraction.submit_transaction
1:NetworkInteraction.wait_for_confirmation
1:NetworkInteraction.wait_for_confirmation {'last_round': 21862378, 'txinfo': {'pool-error': '', 'txn': {'sig': 'ImR1piDoZmWj49NR24RIq7/TlaRg0SGzNPzBClHQvHE7amCnM+DDXjGdbXLvClEiv4SbyYa2Xf64nRefi+ZgCg==', 'txn': {'apap': 'BCAMAQACwAM4B6QCkgFJkQJUAyYJDFBsYXllclhTdGF0ZQxQbGF5ZXJPU3RhdGUJR2FtZVN0YXRlDlBsYXllck9BZGRyZXN

{'app_deployment_txn_log': 'Tic-Tac-Toe application deployed with the application_id: 92488661',
 'start_game_txn_log': 'Game started with the transaction_id: 44ALAH2GZBETY56XP4GDPB3NMJKH64KQPVLOTY4IQE73JZPR7Z2A',
 'play_action_txn_log_X_0': 'X has been put at position 0 in transaction with id: SKLRZXXGEWI6B2JTSOHVBTI2LEWSHDZUOUUO6VFO3KVM4RFYMM4Q',
 'play_action_txn_log_X_1_error': algosdk.error.AlgodHTTPError('TransactionPool.Remember: transaction M35YMK2K3A4XEXZXO5LDPVIAAY5UY4ZYZLDDW5M7UTKEA6BCCQKQ: logic eval error: assert failed pc=424. Details: pc=424, opcodes=app_global_get\n==\nassert\n')}

In [23]:
run('error_position_repeat', reraise=True)

4:GameEngineService.__init__
application_start
initialize_players_logic
play_action_logic
has_player_won
has_player_won
is_tie
money_refund_logic
app_initialization_logic
5:clear_program
4:GameEngineService.deploy_application
application_start
initialize_players_logic
play_action_logic
has_player_won
has_player_won
is_tie
money_refund_logic
app_initialization_logic
5:clear_program
1:NetworkInteraction.compile_program
1:NetworkInteraction.compile_program
5:AppVariables.number_of_int
5:AppVariables.number_of_str
2:ApplicationTransactionRepository.create_application
2:get_default_suggested_params
1:NetworkInteraction.submit_transaction
1:NetworkInteraction.wait_for_confirmation
1:NetworkInteraction.wait_for_confirmation {'last_round': 21862382, 'txinfo': {'pool-error': '', 'txn': {'sig': 'vJazq/NMhkHSHB/11hkP7cuX2q3LRXUQXf3tdtMkGDj4Cx0aGyfAUIbiKvkhKBkF0bUDM7AFOU8qyVztcn4yAQ==', 'txn': {'apap': 'BCAMAQACwAM4B6QCkgFJkQJUAyYJDFBsYXllclhTdGF0ZQxQbGF5ZXJPU3RhdGUJR2FtZVN0YXRlDlBsYXllck9BZGRyZXN

AlgodHTTPError: TransactionPool.Remember: transaction SLHIVDDPQ3U232TQ67ZINCA2SZV2ONKP5C2L7WG6YVOCX3TO7SCA: logic eval error: assert failed pc=448. Details: pc=448, opcodes===
&&
assert
